<a href="https://colab.research.google.com/github/sashi8a/ML_Commodities_VIP_Sp26/blob/main/Bi_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Load in Data, split into train/val/test

In [ ]:
df = pd.read_csv("processed_data.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
train = df[df["Date"] <= pd.Timestamp("2021-12-01")]
val = df[(df["Date"] > pd.Timestamp("2021-12-01")) & (df["Date"] <= pd.Timestamp("2024-12-01"))]
test = df[df["Date"] > pd.Timestamp("2024-12-01")]

In [ ]:
train.head(40)

,Date,Price,sasdate,RPI,W875RX1,CMRMTSPLx,IPFPNSS,USWTRADE,USTRADE,BUSLOANS,...,T5YFFM,T10YFFM,AAAFFM,BAAFFM,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,PPICMM,UMCSENTx
0,1983-03-31,29.27,1983-03-01,6319.274,5489.2,535378.168,55.2854,4478.5,10509.0,392.4259,...,1.22,1.60,2.71,4.49,2.0587,237.7467,1.5361,1.2325,102.1,89.1
1,1983-04-04,29.44,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
2,1983-04-05,29.71,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
3,1983-04-06,29.90,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
4,1983-04-07,30.17,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
5,1983-04-08,30.38,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
6,1983-04-11,30.25,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
7,1983-04-12,30.83,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
8,1983-04-13,30.82,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3
9,1983-04-14,30.82,1983-04-01,6351.636,5514.5,543698.110,55.6149,4491.5,10541.3,391.4906,...,1.40,1.75,2.83,4.46,2.0572,234.7557,1.5722,1.2292,103.3,93.3


In [ ]:
val.head()

,Date,Price,sasdate,RPI,W875RX1,CMRMTSPLx,IPFPNSS,USWTRADE,USTRADE,BUSLOANS,...,T5YFFM,T10YFFM,AAAFFM,BAAFFM,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,PPICMM,UMCSENTx
9809,2021-12-02,66.27,2021-12-01,19062.795,15496.6,1485162.0,98.6614,5852.7,15407.9,2486.9777,...,1.46,1.68,2.85,3.5,0.9191,114.8255,1.3555,1.2622,282.527,67.2
9810,2021-12-03,66.10,2021-12-01,19062.795,15496.6,1485162.0,98.6614,5852.7,15407.9,2486.9777,...,1.46,1.68,2.85,3.5,0.9191,114.8255,1.3555,1.2622,282.527,67.2
9811,2021-12-06,69.30,2021-12-01,19062.795,15496.6,1485162.0,98.6614,5852.7,15407.9,2486.9777,...,1.46,1.68,2.85,3.5,0.9191,114.8255,1.3555,1.2622,282.527,67.2
9812,2021-12-07,71.84,2021-12-01,19062.795,15496.6,1485162.0,98.6614,5852.7,15407.9,2486.9777,...,1.46,1.68,2.85,3.5,0.9191,114.8255,1.3555,1.2622,282.527,67.2
9813,2021-12-08,72.18,2021-12-01,19062.795,15496.6,1485162.0,98.6614,5852.7,15407.9,2486.9777,...,1.46,1.68,2.85,3.5,0.9191,114.8255,1.3555,1.2622,282.527,67.2


In [ ]:
test.tail()

,Date,Price,sasdate,RPI,W875RX1,CMRMTSPLx,IPFPNSS,USWTRADE,USTRADE,BUSLOANS,...,T5YFFM,T10YFFM,AAAFFM,BAAFFM,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,PPICMM,UMCSENTx
10785,2025-08-26,63.25,2025-08-01,20603.322,16654.3,1551287.0,97.7772,6165.9,15605.2,2693.7764,...,-0.57,-0.11,0.98,1.6,0.7961,147.8629,1.3506,1.3834,387.56,55.1
10786,2025-08-27,64.15,2025-08-01,20603.322,16654.3,1551287.0,97.7772,6165.9,15605.2,2693.7764,...,-0.57,-0.11,0.98,1.6,0.7961,147.8629,1.3506,1.3834,387.56,55.1
10787,2025-08-28,64.60,2025-08-01,20603.322,16654.3,1551287.0,97.7772,6165.9,15605.2,2693.7764,...,-0.57,-0.11,0.98,1.6,0.7961,147.8629,1.3506,1.3834,387.56,55.1
10788,2025-08-29,64.01,2025-08-01,20603.322,16654.3,1551287.0,97.7772,6165.9,15605.2,2693.7764,...,-0.57,-0.11,0.98,1.6,0.7961,147.8629,1.3506,1.3834,387.56,55.1
10789,2025-08-31,63.96,2025-08-01,20603.322,16654.3,1551287.0,97.7772,6165.9,15605.2,2693.7764,...,-0.57,-0.11,0.98,1.6,0.7961,147.8629,1.3506,1.3834,387.56,55.1


Base Forecase Model Class template

In [ ]:
"""
Base Template for Forecasting Models

This is a simple template showing what methods your model needs to have.
"""

from abc import ABC, abstractmethod
import numpy as np


class BaseForecastModel(ABC):
    """
    Simple base class for forecasting models.

    Your model should inherit from this and implement all the methods below.

    Example:
        class MyModel(BaseForecastModel):
            def __init__(self, task_type, **params):
                super().__init__(task_type, **params)
                # Your setup code here

            def fit(self, X_train, y_train):
                # Your training code here
                pass

            def predict(self, X):
                # Your prediction code here
                return predictions

            def evaluate(self, X_test, y_test):
                # Your evaluation code here
                return metrics
    """

    def __init__(self, task_type: str, **hyperparameters):
        """
        Initialize your model.

        Args:
            task_type: Either 'regression' or 'classification'
            **hyperparameters: Your model's parameters (e.g., learning_rate=0.01)

        Students should:
            - Store the task_type
            - Store any hyperparameters
            - Initialize your model architecture/components
        """
        self.task_type = task_type
        self.hyperparameters = hyperparameters

    @abstractmethod
    def fit(self, X_train, y_train):
        """
        Train your model on the training data.

        Args:
            X_train: Training features, shape (n_samples, n_features)
            y_train: Training targets, shape (n_samples,)

        You should implement:
            - Data preprocessing if needed
            - Model training logic
            - Store trained parameters/weights

        """
        pass

    @abstractmethod
    def predict(self, X):
        """
        Make predictions on new data.

        Args:
            X: Features to predict on, shape (n_samples, n_features)

        Returns:
            predictions: Your predictions, shape (n_samples,)

        You should implement:
            - Apply same preprocessing as in fit()
            - Generate predictions using trained model
            - Return predictions as numpy array
        """
        pass

    @abstractmethod
    def evaluate(self, X_test, y_test):
        """
        Evaluate model performance on test data.

        Args:
            X_test: Test features
            y_test: True test labels/values

        Returns:
            metrics: Dictionary with metric names and values
                    e.g., {'rmse': 0.5, 'mae': 0.3, 'r2': 0.85}

        You should implement:
            - Get predictions on test data
            - Compute appropriate metrics based on task_type:
              * Regression: RMSE, MAE, R², MAPE
              * Classification: Accuracy, Precision, Recall, F1
            - Return metrics as a dictionary

        """
        pass

    @abstractmethod
    def save(self, filepath: str):
        """
        Save your trained model to a file.

        Args:
            filepath: Path where to save (e.g., 'models/my_model.pkl')

        You should implement:
            - Save model parameters/weights
            - Save any preprocessing parameters (mean, std, etc.)
            - Save hyperparameters
        """
        pass

    @abstractmethod
    def load(self, filepath: str):
        """
        Load a previously saved model.

        Args:
            filepath: Path to the saved model file

        You should implement:
            - Load model parameters/weights
            - Load preprocessing parameters
            - Restore the model to a usable state

        """
        pass









    '''


    """
Example: Simple Linear Regression Model

This shows you exactly what to implement.
"""

import numpy as np
import pickle


class SimpleLinearRegression(BaseForecastModel):
    """
    A simple linear regression model: y = X @ weights + bias

    This is a minimal example showing all required methods.
    """

    def __init__(self, task_type='regression', learning_rate=0.01):
        # Call parent constructor
        super().__init__(task_type=task_type, learning_rate=learning_rate)

        # Initialize model parameters
        self.weights = None
        self.bias = None
        self.learning_rate = learning_rate

    def fit(self, X_train, y_train):
        """Train using closed-form solution."""
        # Add bias column to X
        X_with_bias = np.column_stack([np.ones(len(X_train)), X_train])

        # Closed-form solution: weights = (X^T X)^-1 X^T y
        self.weights = np.linalg.lstsq(X_with_bias, y_train, rcond=None)[0]

        # Split weights into bias and coefficients
        self.bias = self.weights[0]
        self.weights = self.weights[1:]

    def predict(self, X):
        """Make predictions."""
        if self.weights is None:
            raise ValueError("Model not trained! Call fit() first.")

        # y = X @ weights + bias
        predictions = X @ self.weights + self.bias
        return predictions

    def evaluate(self, X_test, y_test):
        """Compute evaluation metrics."""
        predictions = self.predict(X_test)

        # Calculate metrics
        mse = np.mean((y_test - predictions) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - predictions))

        # R² score
        ss_total = np.sum((y_test - np.mean(y_test)) ** 2)
        ss_residual = np.sum((y_test - predictions) ** 2)
        r2 = 1 - (ss_residual / ss_total)

        return {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        }

    def save(self, filepath: str):
        """Save the model."""
        save_dict = {
            'weights': self.weights,
            'bias': self.bias,
            'learning_rate': self.learning_rate,
            'task_type': self.task_type
        }

        with open(filepath, 'wb') as f:
            pickle.dump(save_dict, f)

        print(f"Model saved to {filepath}")

    def load(self, filepath: str):
        """Load the model."""
        with open(filepath, 'rb') as f:
            save_dict = pickle.load(f)

        self.weights = save_dict['weights']
        self.bias = save_dict['bias']
        self.learning_rate = save_dict['learning_rate']

        print(f"Model loaded from {filepath}")


# ============================================================================
# Example of execution on synthetic data
# ============================================================================

if __name__ == "__main__":
    # 1. Create some data
    X_train = np.random.randn(100, 5)
    y_train = X_train @ np.array([1, 2, 3, 4, 5]) + np.random.randn(100) * 0.1

    X_test = np.random.randn(20, 5)
    y_test = X_test @ np.array([1, 2, 3, 4, 5]) + np.random.randn(20) * 0.1

    # 2. Create and train model
    model = SimpleLinearRegression(learning_rate=0.01)
    model.fit(X_train, y_train)

    # 3. Make predictions
    predictions = model.predict(X_test)
    print("Predictions:", predictions[:5])

    # 4. Evaluate
    metrics = model.evaluate(X_test, y_test)
    print("\nMetrics:")
    for name, value in metrics.items():
        print(f"  {name}: {value:.4f}")

    # 5. Save and load
    model.save('linear_model.pkl')

    new_model = SimpleLinearRegression()
    new_model.load('linear_model.pkl')

    # Test loaded model
    new_predictions = new_model.predict(X_test)
    print(f"\nLoaded model works: {np.allclose(predictions, new_predictions)}")

    '''



# Task
Prepare the `train`, `val`, and `test` DataFrames for time series forecasting. This involves:
1. Extracting the 'Price' column as the target variable (`y_train`, `y_val`, `y_test`).
2. Removing 'Date' and 'sasdate' columns from the DataFrames to form the feature sets (`X_train`, `X_val`, `X_test`).
3. Generating sequences of 30 days from the feature sets, where the target for each sequence is the 'Price' value immediately following the sequence. The first 30 prices should be skipped.
4. Converting the resulting feature and target NumPy arrays into PyTorch tensors.
5. Finally, confirm the shapes of the created feature and target tensors for the train, validation, and test sets.

## Prepare Features and Target


Extract the 'Price' column as the target variable and remove 'Date' and 'sasdate' columns from the `train` DataFrame to create the feature set. The resulting feature set will be used to create sequences.


**Reasoning**:
To prepare the feature and target variables as instructed, create `y_train` from the 'Price' column and `X_train` by dropping the specified columns from the `train` DataFrame, resetting their indices, and then displaying the first five rows of both for verification.



In [ ]:
y_train = train['Price'].reset_index(drop=True)
X_train = train.drop(columns=['Date', 'sasdate', 'Price']).reset_index(drop=True)

print("X_train head:")
print(X_train.head())
print("\ny_train head:")
print(y_train.head())

X_train head:
        RPI  W875RX1   CMRMTSPLx  IPFPNSS  USWTRADE  USTRADE  BUSLOANS  \
0  6319.274   5489.2  535378.168  55.2854    4478.5  10509.0  392.4259   
1  6351.636   5514.5  543698.110  55.6149    4491.5  10541.3  391.4906   
2  6351.636   5514.5  543698.110  55.6149    4491.5  10541.3  391.4906   
3  6351.636   5514.5  543698.110  55.6149    4491.5  10541.3  391.4906   
4  6351.636   5514.5  543698.110  55.6149    4491.5  10541.3  391.4906   

       CONSPI  S&P 500  S&P PE ratio  ...  T5YFFM  T10YFFM  AAAFFM  BAAFFM  \
0  112.909566    157.7     11.616356  ...    1.22     1.60    2.71    4.49   
1  112.375069    164.1     12.216329  ...    1.40     1.75    2.83    4.46   
2  112.375069    164.1     12.216329  ...    1.40     1.75    2.83    4.46   
3  112.375069    164.1     12.216329  ...    1.40     1.75    2.83    4.46   
4  112.375069    164.1     12.216329  ...    1.40     1.75    2.83    4.46   

   EXSZUSx   EXJPUSx  EXUSUKx  EXCAUSx  PPICMM  UMCSENTx  
0   2.0587  2

**Reasoning**:
Following the same logic as for the training set, extract the 'Price' column as the target variable and remove 'Date' and 'sasdate' columns from the `val` and `test` DataFrames to create the feature sets for validation and testing.



In [ ]:
y_val = val['Price'].reset_index(drop=True)
X_val = val.drop(columns=['Date', 'sasdate', 'Price']).reset_index(drop=True)

y_test = test['Price'].reset_index(drop=True)
X_test = test.drop(columns=['Date', 'sasdate', 'Price']).reset_index(drop=True)

print("X_val head:")
print(X_val.head())
print("\ny_val head:")
print(y_val.head())
print("\nX_test head:")
print(X_test.head())
print("\ny_test head:")
print(y_test.head())

X_val head:
         RPI  W875RX1  CMRMTSPLx  IPFPNSS  USWTRADE  USTRADE   BUSLOANS  \
0  19062.795  15496.6  1485162.0  98.6614    5852.7  15407.9  2486.9777   
1  19062.795  15496.6  1485162.0  98.6614    5852.7  15407.9  2486.9777   
2  19062.795  15496.6  1485162.0  98.6614    5852.7  15407.9  2486.9777   
3  19062.795  15496.6  1485162.0  98.6614    5852.7  15407.9  2486.9777   
4  19062.795  15496.6  1485162.0  98.6614    5852.7  15407.9  2486.9777   

       CONSPI  S&P 500  S&P PE ratio  ...  T5YFFM  T10YFFM  AAAFFM  BAAFFM  \
0  161.999286  4573.82      28.27503  ...    1.46     1.68    2.85     3.5   
1  161.999286  4573.82      28.27503  ...    1.46     1.68    2.85     3.5   
2  161.999286  4573.82      28.27503  ...    1.46     1.68    2.85     3.5   
3  161.999286  4573.82      28.27503  ...    1.46     1.68    2.85     3.5   
4  161.999286  4573.82      28.27503  ...    1.46     1.68    2.85     3.5   

   EXSZUSx   EXJPUSx  EXUSUKx  EXCAUSx   PPICMM  UMCSENTx  
0   0.91

**Reasoning**:
The next step is to generate sequences of 30 days from the feature sets (`X_train`, `X_val`, `X_test`) and their corresponding target values (`y_train`, `y_val`, `y_test`). Define a helper function to achieve this, ensuring the first 30 prices are skipped for target generation.



In [ ]:
def create_sequences(X, y, sequence_length=30):
    xs, ys = [], []
    # Iterate from sequence_length to the end of the data
    # The target y[i] corresponds to the price after the sequence ending at X[i-1]
    # So, the sequence is X[i - sequence_length : i] and the target is y[i]
    for i in range(sequence_length, len(X)):
        xs.append(X.iloc[i - sequence_length:i].values)
        ys.append(y.iloc[i].item()) # .item() extracts scalar from Series element
    return np.array(xs), np.array(ys)

sequence_length = 30

X_train_seq, y_train_seq = create_sequences(X_train, y_train, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, sequence_length)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"y_val_seq shape: {y_val_seq.shape}")
print(f"X_test_seq shape: {X_test_seq.shape}")
print(f"y_test_seq shape: {y_test_seq.shape}")

X_train_seq shape: (9779, 30, 31)
y_train_seq shape: (9779,)
X_val_seq shape: (759, 30, 31)
y_val_seq shape: (759,)
X_test_seq shape: (162, 30, 31)
y_test_seq shape: (162,)


**Reasoning**:
Now that the sequences have been generated as NumPy arrays, convert them into PyTorch tensors, as required by the task, to prepare them for the PyTorch model. Then print the shapes of the tensors to confirm the conversion.



In [ ]:
X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).view(X_train_seq.shape[0], -1)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).unsqueeze(1)

X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).view(X_val_seq.shape[0], -1)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).view(X_test_seq.shape[0], -1)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).unsqueeze(1)

print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
print(f"X_val_tensor shape: {X_val_tensor.shape}")
print(f"y_val_tensor shape: {y_val_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}")
print(f"y_test_tensor shape: {y_test_tensor.shape}")

X_train_tensor shape: torch.Size([9779, 930])
y_train_tensor shape: torch.Size([9779, 1])
X_val_tensor shape: torch.Size([759, 930])
y_val_tensor shape: torch.Size([759, 1])
X_test_tensor shape: torch.Size([162, 930])
y_test_tensor shape: torch.Size([162, 1])


Define the Bi-*GRU*

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle

class _BiGRUModel(nn.Module):
    """
    Internal PyTorch Module for Bi-GRU with Attention and Skip Connections.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout=0.2):
        super(_BiGRUModel, self).__init__()
        self.hidden_dim = hidden_dim

        # Bidirectional GRU layers
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Attention Mechanism
        # We will use a simple dot-product attention
        self.attention = nn.Linear(hidden_dim * 2, 1)

        # Skip Connection (Residual)
        # We project input to hidden_dim * 2 to match GRU output if dims differ
        self.skip_projection = nn.Linear(input_dim, hidden_dim * 2)

        # Output Layers
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)

        # GRU Output: (batch_size, seq_len, hidden_dim * 2)
        gru_out, _ = self.gru(x)

        # Attention scores
        # (batch_size, seq_len, 1)
        attn_scores = self.attention(gru_out)
        attn_weights = torch.softmax(attn_scores, dim=1)

        # Context vector (weighted sum of GRU outputs)
        # (batch_size, hidden_dim * 2)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)

        # Skip Connection
        # We take the last input of the sequence for the skip connection
        # (batch_size, input_dim) -> (batch_size, hidden_dim * 2)
        skip_input = x[:, -1, :]
        skip_out = self.skip_projection(skip_input)

        # Combine Context and Skip
        combined = context_vector + skip_out

        # Output
        out = self.fc(combined)
        return out

In [ ]:
class BiGRUAttentionModel(BaseForecastModel):
    """
    Bi-GRU with Attention and Skip Connections for Time Series Forecasting.
    """
    def __init__(self, task_type='regression', input_dim=930, hidden_dim=24, num_layers=4, dropout=0.2, learning_rate=0.003, epochs=10, batch_size=32):
        super().__init__(task_type=task_type, input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout, learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

        self.model = None
        self.criterion = None
        self.optimizer = None

        # Device configuration
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    def fit(self, X_train, y_train):

        if not self.model:
            self.model = _BiGRUModel(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            output_dim=1, # output dimension is 1 since we are predicting one value
            num_layers=self.num_layers,
            dropout=self.dropout
            ).to(self.device)

            self.criterion = nn.MSELoss()
            self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        """
        Train the model using PyTorch training loop.
        X_train shape: (num_samples, seq_len, input_dim)
        y_train shape: (num_samples,)
        """


        dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            batch_losses = []
            for batch_X, batch_y in dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                self.optimizer.step()
                batch_losses.append(loss.item())

            # Optional: Print progress
            if (epoch + 1) % max(1, self.epochs // 10) == 0:
                print(f"Epoch [{epoch+1}/{self.epochs}], Loss: {np.mean(batch_losses):.4f}")

    def predict(self, X):
        """
        Make predictions.
        X shape: (num_samples, seq_len, input_dim)
        """
        if self.model is None:
             raise ValueError("Model not trained! Call fit() first.")

        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)

        with torch.no_grad():
            predictions = self.model(X_tensor).cpu().numpy()

        return predictions.flatten()

    def evaluate(self, X_test, y_test):
        """
        Evaluate the model.
        """
        predictions = self.predict(X_test)

        mse = np.mean((y_test - predictions) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - predictions))

        # R² score
        ss_total = np.sum((y_test - np.mean(y_test)) ** 2)
        ss_residual = np.sum((y_test - predictions) ** 2)
        r2 = 1 - (ss_residual / ss_total) if ss_total != 0 else 0

        return {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        }

    def save(self, filepath: str):
        """
        Save model state and hyperparameters.
        """
        state = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'hyperparameters': self.hyperparameters,
             # Also save the explicitly stored attributes in case they were modified or needed
            'config': {
                'input_dim': self.input_dim,
                'hidden_dim': self.hidden_dim,
                'num_layers': self.num_layers,
                'dropout': self.dropout,
                'learning_rate': self.learning_rate,
                'epochs': self.epochs,
                'batch_size': self.batch_size
            }
        }
        torch.save(state, filepath)
        print(f"Model saved to {filepath}")

    def load(self, filepath: str):
        """
        Load model state.
        """
        checkpoint = torch.load(filepath, map_location=self.device)

        # Restore hyperparameters
        config = checkpoint.get('config', {})
        self.input_dim = config.get('input_dim', self.input_dim)
        self.hidden_dim = config.get('hidden_dim', self.hidden_dim)
        self.num_layers = config.get('num_layers', self.num_layers)
        self.dropout = config.get('dropout', self.dropout)
        self.learning_rate = config.get('learning_rate', self.learning_rate)
        # Note: epochs and batch_size are mainly for training, so restoring them is optional but good practice
        self.epochs = config.get('epochs', self.epochs)
        self.batch_size = config.get('batch_size', self.batch_size)

        # Rebuild model matches the saved config
        self._build_model()

        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(f"Model loaded from {filepath}")

In [ ]:
if __name__ == '__main__':
    # Synthetic Data Test
    print("Generating synthetic data...")
    # Generate random sequences: (num_samples, seq_len, input_dim)
    N = 1000
    seq_len = 10
    input_dim = 1

    X = np.random.randn(N, seq_len, input_dim)
    # Simple target: sum of the sequence plus some noise
    y = np.sum(X, axis=1).flatten() + np.random.randn(N) * 0.1

    # Split
    split = int(0.8 * N)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    print("Initializing model...")
    model = BiGRUAttentionModel(
        input_dim=input_dim,
        hidden_dim=32,
        num_layers=1,
        epochs=5,
        batch_size=32
    )

    print("Training model...")
    model.fit(X_train, y_train)

    print("Evaluating model...")
    metrics = model.evaluate(X_test, y_test)
    print("Metrics:", metrics)

    print("Testing Save/Load...")
    model.save('test_bi_gru_model.pth')

    loaded_model = BiGRUAttentionModel(input_dim=input_dim, hidden_dim=32, num_layers=1)
    loaded_model.load('test_bi_gru_model.pth')

    # Check predictions match
    pred_orig = model.predict(X_test[:5])
    pred_load = loaded_model.predict(X_test[:5])

    print("Original Predictions:", pred_orig)
    print("Loaded Predictions:  ", pred_load)
    print("Match:", np.allclose(pred_orig, pred_load))
